# The N-Queens Puzzle (Strict NNF)

This notebook solves the N-Queens puzzle using the strict NNF type system. It constructs clauses using `NNFNegation` objects and uses the adapted Davis-Putnam solver.

In [ ]:
import { RecursiveSet } from './recursive-set';

In [ ]:
import { Variable, LogicParser } from './propositional-logic-parser';
import { Literal, Clause, CNF, NNFNegation } from './cnf';
import * as DP from './davis-putnam-jw';

### Variable Definition
Variables are strings representing the position `Q<row,col>`.

In [ ]:
function varName(row: number, col: number): Variable {
    return `Q<${row},${col}>`;
}

### Constraint Generation

The function `atMostOne` is updated to create clauses using **strict NNF negation** (`new NNFNegation(p)`) instead of tuples.

In [ ]:
function atMostOne(S: RecursiveSet<Variable>): CNF {
    const arr = Array.from(S);
    const clauses: Clause[] = [];
    for (let i = 0; i < arr.length; i++) {
        for (let j = i + 1; j < arr.length; j++) {
            const p = arr[i];
            const q = arr[j];
            // create the clause { ¬p, ¬q }
            const clause = RecursiveSet.fromArray<Literal>([
                new NNFNegation(p), 
                new NNFNegation(q)
            ]);
            clause.freeze();
            clauses.push(clause as Clause);
        }
    }
    return RecursiveSet.fromArray<Clause>(clauses) as CNF;
}

We define helpers for rows, columns, and diagonals using the updated `atMostOne`.

In [ ]:
function atMostOneInRow(row: number, n: number): CNF {
    const VarsInRow = new RecursiveSet<Variable>();
    for (let col = 1; col <= n; col++) {
        VarsInRow.add(varName(row, col));
    }
    return atMostOne(VarsInRow);
}

function oneInColumn(col: number, n: number): CNF {
    const VarsInColumn = new RecursiveSet<Literal>();
    for (let row = 1; row <= n; row++) {
        VarsInColumn.add(varName(row, col));
    }
    VarsInColumn.freeze();
    
    const result = new RecursiveSet<Clause>();
    result.add(VarsInColumn);
    return result as CNF;
}

function atMostOneInFallingDiagonal(k: number, n: number): CNF {
    const Vars = new RecursiveSet<Variable>();
    for (let row = 1; row <= n; row++) {
        for (let col = 1; col <= n; col++) {
            if (row - col === k) Vars.add(varName(row, col));
        }
    }
    return atMostOne(Vars);
}

function atMostOneInRisingDiagonal(k: number, n: number): CNF {
    const Vars = new RecursiveSet<Variable>();
    for (let row = 1; row <= n; row++) {
        for (let col = 1; col <= n; col++) {
            if (row + col === k) Vars.add(varName(row, col));
        }
    }
    return atMostOne(Vars);
}

### Combining All Constraints

In [ ]:
function allClauses(n: number): CNF {
    const result = new RecursiveSet<Clause>();
    
    // Helper to add a set of clauses to the result
    const add = (s: CNF) => { 
        for (const c of s) result.add(c); 
    };

    for (let row = 1; row <= n; row++) {
        add(atMostOneInRow(row, n));
    }
    for (let col = 1; col <= n; col++) {
        add(oneInColumn(col, n));
    }
    for (let k = -(n - 2); k <= n - 2; k++) {
        add(atMostOneInFallingDiagonal(k, n));
    }
    for (let k = 3; k <= 2 * n; k++) {
        add(atMostOneInRisingDiagonal(k, n));
    }
    
    return result as CNF;
}

### Solving the Problem
We generate the clauses and pass them to the adapted `DP.solve` function.

In [ ]:
function queens(n: number): CNF | null {
    const Clauses = allClauses(n);
    const Solution = DP.solve(Clauses);
    
    const EmptyClause = new RecursiveSet<Literal>();
    EmptyClause.freeze();
    
    if (Solution.has(EmptyClause)) {
        console.log(`The problem is not solvable for ${n} queens!`);
        return null;
    }
    return Solution;
}

In [ ]:
console.time('queens-16');
const Solution = queens(16);
console.timeEnd('queens-16');

### Visualization

We update `removeNegativeLiterals` to detect negative literals using `instanceof NNFNegation`.

In [ ]:
function removeNegativeLiterals(Solution: CNF): RecursiveSet<Variable> {
    const Result = new RecursiveSet<Variable>();
    for (const C of Solution) {
        for (const lit of C) {
            // UPDATE: Check if literal is a positive variable (string)
            // by ensuring it is NOT an NNFNegation instance.
            if (!(lit instanceof NNFNegation)) {
                Result.add(lit as Variable);
            }
        }
    }
    return Result;
}

function extractRowCol(varName: string): [string, string] {
    const left = varName.indexOf('<');
    const comma = varName.indexOf(',');
    const right = varName.indexOf('>');
    const row = varName.substring(left + 1, comma);
    const col = varName.substring(comma + 1, right);
    return [row, col];
}

function transform(Solution: CNF): Record<number, number> {
    const positiveLiterals = removeNegativeLiterals(Solution);
    const Result: Record<number, number> = {};
    for (const name of positiveLiterals) {
        const [row, col] = extractRowCol(name);
        Result[parseInt(row, 10)] = parseInt(col, 10);
    }
    return Result;
}

In [ ]:
import { display } from 'tslab';

function showSolution(Solution: CNF, width = "50%") {
    const transformed = transform(Solution);
    const n = Object.keys(transformed).length;
    if (n === 0) {
        console.log("No solution found to display.");
        return;
    }
    
    const boardArray: string[][] = Array.from({ length: n }, () => Array(n).fill(''));
    for (let row = 1; row <= n; row++) {
        const col = transformed[row];
        if (col !== undefined) {
            if (row - 1 < n && col - 1 < n) {
                boardArray[row - 1][col - 1] = '♕';
            }
        }
    }
    let html = `<div style="display:grid; grid-template-columns:repeat(${n}, 1fr); width:${width}; aspect-ratio: 1/1; border: 2px solid black;">`;
    for (let row = 0; row < n; row++) {
        for (let col = 0; col < n; col++) {
            const piece = boardArray[row][col];
            const bgColor = (row + col) % 2 === 0 ? '#f0d9b5' : '#b58863'; 
            html += `<div style="
                display: flex; 
                align-items: center; 
                justify-content: center; 
                font-size: 2em; 
                background-color:${bgColor};
                color: black;
                ">${piece}</div>`;
        }
    }
    html += `</div>`;
    display.html(html);
}

In [ ]:
if (Solution) {
    showSolution(Solution, "50%");
}